In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv("../data/train_preprocessed.csv")
train_x = train.drop(["target"], axis=1)
train_y = train["target"]

test_x = pd.read_csv("../data/test_preprocessed.csv")
test_x = test_x.dropna(how="any")

In [5]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]

tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [6]:
import lightgbm as lgb
from sklearn.metrics import log_loss

class Model:
    def __init__(self, params=None):
        self.model = None
        if params is None:
           self.params = {}
        else:
            self.params = params

    def fit(self, tr_x, tr_y, va_x, va_y):
        params = {'objective': 'binary', 'seed': 71, 'verbose': 0, 'metrics': 'binary_logloss'}
        num_round = 100
        params.update(self.params)

        dtrain = lgb.Dataset(tr_x, tr_y)
        dvalid = lgb.Dataset(va_x, va_y)

        self.model = lgb.train(
            params, dtrain, num_boost_round=num_round, valid_names=["train", "valid"], valid_sets=[dtrain, dvalid]
        )
    def perdict(self, x):
        pred = self.model.predict(x)
        return pred
